In [2]:
import random
import pandas as pd
import numpy as np
from glob import glob
import os
import cv2
import json
import copy
from sklearn.model_selection import train_test_split

import time
import datetime
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

In [3]:
# origin_path는 원천데이터(이미지데이터)가 저장되어있는 경로로 지정해야합니다.
# label_path는 라벨링데이터가 저장되어있는 경로로 지정해야합니다.
origin_path='./4.Sample/01.원천데이터'
label_path='./4.Sample/02.라벨링데이터'

#시드 고정
random_seed = 42
random.seed(random_seed)

# 데이터 경로 포함 프레임 구축하기

In [4]:
img_list=[] #이미지 파일명 까지 경로
img_id=[] #이미지 파일명
label_list=[] #라벨링 파일명 까지 경로

#이미지 파일명 경로 및 이미지 파일명 Get
for i in os.listdir(origin_path):
    for j in os.listdir(origin_path+'/'+i):
        for k in os.listdir(origin_path+'/'+i+'/'+j):
            if (k=='02.애벌레') or (k=='12.생애이슈(백묵병)'): #02.애벌레=음성, 12.생애이슈(백묵병)=양성
                img_id.extend(os.listdir(origin_path+'/'+i+'/'+j+'/'+k))
                img_list.extend(glob(origin_path+'/'+i+'/'+j+'/'+k+'/*'))   
img_list=[path.replace('\\','/') for path in img_list]

#라벨링 파일명 및 경로 Get
for i in os.listdir(label_path):
    for j in os.listdir(label_path+'/'+i):
        for k in os.listdir(label_path+'/'+i+'/'+j):
            if (k=='02.애벌레') or (k=='12.생애이슈(백묵병)'):
                label_list.extend(glob(label_path+'/'+i+'/'+j+'/'+k+'/*'))
label_list=[path.replace('\\','/') for path in label_list]

#데이터 프레임 구축
data={'이미지명':img_id,'이미지경로':img_list,'라벨링경로':label_list}
data=pd.DataFrame(data)

In [5]:
#Class List
class_list = ['LA_NA', 'AA_NA'] #음성, 양성

#라벨링값 채우기
label=[]
for i in data.index:
    img_name=data['이미지명'][i]
    matching=[word for word in class_list if word in img_name]
    label.append(class_list.index(matching[0]))
    
data['라벨값']=label

# 데이터 분할 및 분석 데이터 셋 구축

In [6]:
# train_test_split 의 stratify 이용해서 분할
train, val = train_test_split(data, test_size = 0.1, random_state = 42, stratify = data['라벨값'])
train, test= train_test_split(train,test_size = 0.1, random_state = 42, stratify = train['라벨값'])

In [7]:
train

,이미지명,이미지경로,라벨링경로,라벨값
61,06_1_R_AA_NA_20221102_04_0626.jpg,./4.Sample/01.원천데이터/01.실제데이터/01.Bounding Box/1...,./4.Sample/02.라벨링데이터/01.실제데이터/01.Bounding Box/...,1
147,01_1_R_LA_NA_20220913_01_0318.jpg,./4.Sample/01.원천데이터/01.실제데이터/02.Polygon/02.애벌레...,./4.Sample/02.라벨링데이터/01.실제데이터/02.Polygon/02.애벌...,0
29,01_1_R_LA_NA_20220720_07_0532.jpg,./4.Sample/01.원천데이터/01.실제데이터/01.Bounding Box/0...,./4.Sample/02.라벨링데이터/01.실제데이터/01.Bounding Box/...,0
185,01_1_D_LA_NA_20220817_12_0057.jpg,./4.Sample/01.원천데이터/02.파괴데이터/01.Bounding Box/0...,./4.Sample/02.라벨링데이터/02.파괴데이터/01.Bounding Box/...,0
125,01_1_R_LA_NA_20220907_01_0433.jpg,./4.Sample/01.원천데이터/01.실제데이터/02.Polygon/02.애벌레...,./4.Sample/02.라벨링데이터/01.실제데이터/02.Polygon/02.애벌...,0
...,...,...,...,...
160,01_1_D_LA_NA_20220812_10_0355.jpg,./4.Sample/01.원천데이터/02.파괴데이터/01.Bounding Box/0...,./4.Sample/02.라벨링데이터/02.파괴데이터/01.Bounding Box/...,0
58,06_1_R_AA_NA_20221102_03_0412.jpg,./4.Sample/01.원천데이터/01.실제데이터/01.Bounding Box/1...,./4.Sample/02.라벨링데이터/01.실제데이터/01.Bounding Box/...,1
198,01_1_D_LA_NA_20220818_04_0246.jpg,./4.Sample/01.원천데이터/02.파괴데이터/01.Bounding Box/0...,./4.Sample/02.라벨링데이터/02.파괴데이터/01.Bounding Box/...,0
57,06_1_R_AA_NA_20221102_03_0333.jpg,./4.Sample/01.원천데이터/01.실제데이터/01.Bounding Box/1...,./4.Sample/02.라벨링데이터/01.실제데이터/01.Bounding Box/...,1


In [8]:
trans_resize = transforms.Resize((224, 224))
trans_tensor = transforms.ToTensor()
def dataset(data):
    data1=[]
    for i in tqdm(data.index):
        with open(data['라벨링경로'][i], 'r', encoding='UTF-8') as f:
            jsonfile=json.load(f)
        if jsonfile['INFO']['DATASET_DETAIL']=='Bounding Box':
            for j in jsonfile['ANNOTATION_INFO']:
                x_data = trans_tensor(trans_resize(Image.open(data['이미지경로'][i]).crop((j['XTL'], j['YTL'], j['XBR'], j['YBR']))))
                y_data= data['라벨값'][i]
                data_list=[[x_data,y_data,jsonfile['IMAGE']['IMAGE_FILE_NAME']+'_'+str((jsonfile['ANNOTATION_INFO'].index(j))+1)]]
                data1.extend(data_list)
            
        elif jsonfile['INFO']['DATASET_DETAIL']=='Polygon':
            for m in jsonfile['ANNOTATION_INFO']:
                XTL = min(m['POLYGON'][0::2])
                XBR = max(m['POLYGON'][0::2])
                YTL = min(m['POLYGON'][0::2])
                YBR = max(m['POLYGON'][0::2])
                x_data = trans_tensor(trans_resize(Image.open(data['이미지경로'][i]).crop((XTL, YTL, XBR, YBR))))
                y_data = data['라벨값'][i]
                data_list=[[x_data,y_data,jsonfile['IMAGE']['IMAGE_FILE_NAME']+'_'+str((jsonfile['ANNOTATION_INFO'].index(m))+1)]]
                data1.extend(data_list)
    print(f'{len(data1)}개 업로드 완료')
    return data1

In [9]:
traindataset = dataset(data=train)
valdataset = dataset(data=val)
testdataset = dataset(data=test)

100%|████████████████████████████████████████████████████████████████████████████████| 162/162 [00:11<00:00, 14.33it/s]


1282개 업로드 완료


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 17.36it/s]


137개 업로드 완료


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:01<00:00, 17.29it/s]

125개 업로드 완료


# Train Start

In [15]:
class SEBlock(nn.Module):
    def __init__(self, in_channel, r=1):
        super().__init__()
        
        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        
        self.excitation = nn.Sequential(
            nn.Linear(in_channel, (in_channel//r)),
            nn.ReLU(),
            nn.Linear((in_channel//r), in_channel),
            nn.Sigmoid()
            )
        
    def forward(self, x):
        block_input = x
        x = self.squeeze(x)
        x = x.view(x.size(0), -1)
        x = self.excitation(x)
        x = x.view(x.size(0), x.size(1), 1, 1)
        
        return x * block_input

In [16]:
class Inception(nn.Module):
    '''
    in_channels (integer)   : Input dataset 의 channel 수, default = 3
    use_auxiliary (boolean) : Auxiliary의 사용 여부, default = True
    num_classes (integer)   : Input dataset 의 class(label)의 개수, default = 10
    
    GoogleNet 모형의 전체 구조 구현
    '''
    def __init__(self,num_classes, in_channels=3, use_auxiliary=True):
        super(Inception, self).__init__()
        
        self.conv1 = ConvBlock(in_channels, 64, kernel_size=7, stride=2, padding=3)
        self.conv2 = ConvBlock(64, 192, kernel_size=3, stride=1, padding=1)
        
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        
        self.dropout = nn.Dropout(0.4)
        self.linear = nn.Linear(1024, num_classes)
        
        self.use_auxiliary = use_auxiliary
        if use_auxiliary:
            self.auxiliary4a = Auxiliary(512, num_classes)
            self.auxiliary4d = Auxiliary(528, num_classes)
        
        self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)

    def forward(self, x):
        y = None
        z = None
        
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool(x)
        
        x = self.inception4a(x)
        if self.training and self.use_auxiliary:
            y = self.auxiliary4a(x)
        
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        if self.training and self.use_auxiliary:
            z = self.auxiliary4d(x)
        
        x = self.inception4e(x)
        x = self.maxpool(x)
        
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        
        x = self.linear(x)
        
        return x, y, z

In [17]:
class ConvBlock(nn.Module):
    '''
    in_channels (integer)   : Input dataset 의 channel 수
    out_channels (integer)  : Output dataset 의 channel 수
    kernel_size (integer) : Convolution Layer의 kernel 크기
    
    GoogleNet 모형의 Inception Module에 사용되는 Convolution Block을 구현
    '''
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

In [18]:
class InceptionBlock(nn.Module):
    '''
    in_channels (integer)   : Input dataset 의 channel 수
    num_1x1(integer)        : Filter의 크기가 1X1인 Convolution Block(self.one_by_one)의 출력 channel 수
    num_3x3_red(integer)    : Filter의 크기가 1X1인 Convolution Block(self.tree_by_three_red)의 출력 channel 수
    num_3x3(integer)        : Filter의 크기가 3X3인 Convolution Block(self.tree_by_three)의 출력 channel 수
    num_5x5_red(integer)    : Filter의 크기가 1X1인 Convolution Block(self.five_by_five_red)의 출력 channel 수
    num_5x5(integer)        : Filter의 크기가 5X5인 Convolution Block(self.five_by_five)의 출력 channel 수
    num_pool_proj(integer)  : Filter의 크기가 1X1인 Convolution Block(self.pool_proj)의 출력 channel 수
    
    GoogleNet 모형의 Inception Module을 구현
    '''
    def __init__(self, in_channels, num_1x1, num_3x3_red, num_3x3, num_5x5_red, num_5x5, num_pool_proj):
        super(InceptionBlock, self).__init__()
        
        self.one_by_one = ConvBlock(in_channels, num_1x1, kernel_size=1)
        
        self.tree_by_three_red = ConvBlock(in_channels, num_3x3_red, kernel_size=1)  
        self.tree_by_three = ConvBlock(num_3x3_red, num_3x3, kernel_size=3, padding=1)
        
        self.five_by_five_red = ConvBlock(in_channels, num_5x5_red, kernel_size=1)
        self.five_by_five = ConvBlock(num_5x5_red, num_5x5, kernel_size=5, padding=2)
        
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.pool_proj = ConvBlock(in_channels, num_pool_proj, kernel_size=1)
        
        self.se_block = SEBlock(in_channel=(num_1x1+num_3x3+num_5x5+num_pool_proj), r=16)
         
    def forward(self, x):
        x1 = self.one_by_one(x)
        
        x2 = self.tree_by_three_red(x)
        x2 = self.tree_by_three(x2)
        
        x3 = self.five_by_five_red(x)
        x3 = self.five_by_five(x3)
        
        x4 = self.maxpool(x)
        x4 = self.pool_proj(x4)
        
        x = torch.cat([x1, x2, x3, x4], 1)
        x = self.se_block(x)
        return x

In [19]:
class Auxiliary(nn.Module):
    '''
    in_channels (integer)   : Input dataset 의 channel 수          
    num_classes (integer)   : Input dataset 의 class(label)의 개수 
    
    GoogleNet 모형에서 train dataset 학습시 사용되는 Auxiliary Classifier를 구현
    '''
    def __init__(self, in_channels, num_classes):
        super(Auxiliary, self).__init__()
        self.avgpool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv1x1 = ConvBlock(in_channels, 128, kernel_size=1)
        
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        
        self.dropout = nn.Dropout(0.7)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.avgpool(x)
        x = self.conv1x1(x)
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [20]:
class GoogleNetModel(nn.Module):
    '''
    train_dataset (dataset)         : 모형 학습에서 train dataset 로 사용할 데이터 셋
    val_dataset (dataset)           : 모형 학습에서 validation dataset 로 사용할 데이터 셋
    epoch (integer)              : 학습을 반복할 횟수
    learning_rate (float)        : 모형 학습에서 사용할 learning_rate
    batch_size (integer)         : 한 번 학습에 사용할 이미지의 개수
    loader_num_workers (integer) : DataLoader에서 사용할 코어의 개수
        
    GoogleNet 모형을 학습시키는 일련의 과정을 구현
    '''
    def __init__(self, train_dataset, val_dataset, num_classes, epoch, learning_rate, batch_size, loader_num_workers):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset   
        self.num_classes = num_classes
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.loader_num_workers = loader_num_workers
        self.patience_i = int(min(np.ceil(epoch*0.1), 10))
        
        
    def GoogleNetTrain(self):
        model = Inception(num_classes = self.num_classes).to(self.device)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=self.learning_rate, weight_decay=1e-4)
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=self.patience_i, verbose=True)
        
        train_loader = DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.loader_num_workers)
        val_loader = DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.loader_num_workers)
        dataloaders = {"train": train_loader, "val": val_loader}
                
        since = time.time()
        train_acc_history = []
        train_loss_history = []
        val_acc_history = []
        val_loss_history = []
        best_model_wts = copy.deepcopy(model.state_dict())
        best_acc = 0.0
        use_auxiliary=True

        for i in range(self.epoch):
            epoch_time = time.time()
            print('Epoch {}/{}'.format(i+1, self.epoch))
            print('-' * 10)

            for phase in ['train', 'val']: # Each epoch has a training and validation phase
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                for inputs, labels in dataloaders[phase]: # Iterate over dataset

                    inputs = inputs.to(self.device)

                    labels = labels.to(self.device)

                    optimizer.zero_grad() # Zero the parameter gradients

                    with torch.set_grad_enabled(phase == 'train'): # Forward. Track history if only in train

                        if phase == 'train': # Backward + optimize only if in training phase
                            if use_auxiliary:
                                outputs, aux1, aux2 = model(inputs)
                                loss = criterion(outputs, labels) + 0.3 * criterion(aux1, labels) + 0.3 * criterion(aux2, labels)
                            else:
                                outputs, _, _ = model(inputs)
                                loss = criterion(outputs, labels)

                            _, preds = torch.max(outputs, 1)
                            loss.backward()
                            optimizer.step()

                        if phase == 'val':
                            outputs, _, _ = model(inputs)
                            loss = criterion(outputs, labels)
                            _, preds = torch.max(outputs, 1)

                    # Statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / len(dataloaders[phase].dataset)

                if phase == 'val': # Adjust learning rate based on val loss
                    lr_scheduler.step(epoch_loss)

                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
                
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                e_time = time.time() - epoch_time

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_epoch = i+1
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                if phase == 'val':
                    val_acc_history.append(float(epoch_acc))
                    val_loss_history.append(float(epoch_loss))
                if phase == 'train':
                    train_acc_history.append(float(epoch_acc))
                    train_loss_history.append(float(epoch_loss))

            print('Epoch Time : {:.0f}m {:.0f}s'.format(e_time//60, e_time%60))
            print()

        time_elapsed = time.time() - since
        
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best Model is {0}'.format(best_epoch))
        print('Best val Acc: {:4f}'.format(best_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
        
        return model, self.epoch, train_loss_history, train_acc_history, val_loss_history, val_acc_history

# Model Training 진행

In [21]:
model, epoch, train_loss_history, train_acc_history, val_loss_history, val_acc_history = GoogleNetModel(train_dataset = [i[0:2] for i in traindataset],
                                                                                                        val_dataset = [i[0:2] for i in valdataset],
                                                                                                        num_classes = 2, epoch = 15, 
                                                                                                        learning_rate = 0.05, batch_size = 4,
                                                                                                        loader_num_workers = 2).GoogleNetTrain()
if (os.path.exists('./models')==False):
    os.makedirs('./models')
torch.save(model, f'./models/GoogleNet_SE_Lifeissue_AA_Final.pt')
torch.save(model.state_dict(), f'./models/GoogleNet_SE_stat_Lifeissue_AA_Final.pt')

Epoch 1/15
----------
train Loss: 11.1613 Acc: 0.7839
val Loss: 3.0664 Acc: 0.2920
Epoch Time : 0m 27s

Epoch 2/15
----------
train Loss: 0.8812 Acc: 0.8362
val Loss: 0.1619 Acc: 0.9489
Epoch Time : 0m 22s

Epoch 3/15
----------


KeyboardInterrupt: 

# Model Test 진행

In [19]:
random_seed=42
try:
    if torch.cuda.is_available():
        device = torch.device('cuda')
        
    else:
        device = torch.device('cpu')

except:
    device = torch.device('cpu')
    
Image.MAX_IMAGE_PIXELS = None
try:
    torch.backends.cudnn.deterministic = True    
    torch.backends.cudnn.benchmark = False 
except:
    pass
np.random.seed(random_seed)
try:
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
except:
    pass

#### Test Accuracy for classes
model = torch.load('./models/GoogleNet_Lifeissue_AA_Final.pt', map_location=device)
model.load_state_dict(torch.load('./models/GoogleNet_stat_Lifeissue_AA_Final.pt', map_location=device))
model = model.to(device)

test_loader = DataLoader(testdataset, batch_size=8, shuffle=False, num_workers=2)

label_list = list()
prediction_list = list()

#### Print Result
print('GoogleNet Test 결과 출력', '(', datetime.datetime.now(), ')')


classes = ['LA_NEG','LA_POS']
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

print(f'{device}로 결과를 출력중입니다. cpu로 계산할 경우 시간이 오래 걸리니 기다려주세요.')

model.eval()
#logger.info('GoogleNet 모델의 Test를 진행합니다.')
with torch.no_grad():
    for data in test_loader:
        images, labels, _ = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predictions = torch.max(outputs[0], 1)    

        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1
            label_list.append(label.tolist())
            prediction_list.append(prediction.tolist())
mean = 0
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]

    print(f'Accuracy for class : {classname:7s} is {accuracy:.2f} %', '(', datetime.datetime.now(), ')')
    mean += accuracy
print(f'\nTotal classes : {mean/len(classes):.2f} %', '(', datetime.datetime.now(), ')')

GoogleNet Test 결과 출력 ( 2023-12-27 11:34:40.678927 )
cuda로 결과를 출력중입니다. cpu로 계산할 경우 시간이 오래 걸리니 기다려주세요.
Accuracy for class : LA_NEG  is 100.00 % ( 2023-12-27 11:34:43.304009 )
Accuracy for class : LA_POS  is 100.00 % ( 2023-12-27 11:34:43.305011 )

Total classes : 100.00 % ( 2023-12-27 11:34:43.305011 )


# 유효성 증빙서류 

1. 개별 결과값

2. confusion matrix

In [30]:
# 1. 개별 결과값 Test data set

test_ID=[]
for i in range(len(testdataset)):
    ID=testdataset[i][2]
    test_ID.append(ID)

    
test_ID=pd.DataFrame(test_ID)
label_data=pd.DataFrame(label_list)
pred_data=pd.DataFrame(prediction_list)

df_list=pd.concat([test_ID,label_data],axis=1)
df_list=pd.concat([df_list,pred_data],axis=1)
df_list.columns=['file_name','label','prediction']
df_list

# 문제당 개별 결과값 저장
df_list.to_csv('./filelist'+'문제 당 개별 결과값.csv',encoding='cp949',index=False)

In [32]:
# 2. confusion matrix

cf=confusion_matrix(label_list, prediction_list)

cf=pd.DataFrame(cf).rename(index={0:'True_백묵병음성',1:'True_백묵병양성'}
                             ,columns={0:'Pred_백묵병음성',1:'pred_백묵병양성'})
#계산할 때 사용된 값 저장 (Confusion matrix 기반 TP, FP, TN, FN)
cf.to_csv('./filelist'+'Confusion Matrix 기반 TP, FP, TN, FN(백묵병 분류 모델).csv',encoding='cp949')